# ISL Dataprep
## Run trimming, mediapipe pose estimation, metadata etc on downsampled videos

In [ ]:
!nproc

8


## Set up data

Data Source: Lambda filesystem
  - bucket name: 921c05b3-8e02-4c0b-9623-d1de39939163
  - region: us-east-3


AWS creds needed:
  - AWS_ACCESS_KEY_ID=9P7Q2ADQR5WPGY83629B
  - AWS_SECRET_ACCESS_KEY=*********
  - AWS_REGION=us-east-3
  - S3_ENDPOINT_URL="https://files.us-east-3.lambda.ai"

Change the bucket and credentials when using a different instance. These are given as pattern reference.

In [ ]:
!pip install awscli boto3

In [ ]:
! aws --version

aws-cli/1.42.34 Python/3.12.11 Linux/6.1.123+ botocore/1.40.34


In [ ]:
import os
os.environ['AWS_ACCESS_KEY_ID'] = '**********'
os.environ['AWS_SECRET_ACCESS_KEY'] = '*********'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-3'

In [ ]:
! aws configure

AWS Access Key ID [None]: C0INV916QMS8WB9F104E
AWS Secret Access Key [None]: 5aQ8EFIaafQmlwOQHehpbNnhPCinepC+vuRmnE/H
Default region name [None]: us-east-3
Default output format [None]: 


In [ ]:
!echo "[default]" > ~/.aws/config
!echo "region =us-east-3 " >> ~/.aws/config
!echo "endpoint-url =https://files.us-east-3.lambda.ai " >> ~/.aws/config
!echo "" >> ~/.aws/config


!echo "request_checksum_calculation =when_required " >> ~/.aws/config
!echo "response_checksum_validation =when_required " >> ~/.aws/config
!echo "" >> ~/.aws/config


In [ ]:
cat ~/.aws/config

[default]
region =us-east-3 
endpoint-url =https://files.us-east-3.lambda.ai 

request_checksum_calculation =when_required 
response_checksum_validation =when_required 



In [ ]:
!aws s3 ls s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/islNTvideos/ --endpoint-url https://files.us-east-3.lambda.ai

In [ ]:
!aws s3 cp s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/islNTvideos/ /content/islNTvideos --recursive --endpoint-url https://files.us-east-3.lambda.ai

In [ ]:
! df -h /

Filesystem      Size  Used Avail Use% Mounted on
overlay         226G   40G  187G  18% /


In [ ]:
ls /content/

islNTvideos/  sample_data/


In [ ]:
ls /content/islNTvideos/*/*/*.MP4 | wc

    855    1696   40930


In [ ]:
ls /content/islNTvideos/*/*.MP4 | wc

     17      34     744


In [ ]:
ls /content/islNTvideos/*/*/*.mp4 | wc

      1       3      55


In [ ]:
mkdir /content/islNTvideos/Jude/CH-1

In [ ]:
mv /content/islNTvideos/Jude/*.MP4 /content/islNTvideos/Jude/CH-1/

In [ ]:
mv /content/islNTvideos/Hebrews/*.MP4 /content/islNTvideos/Hebrews/CH-13/

In [ ]:
mv '/content/islNTvideos/1-Thessalonians/3 Ch/V- 11-13.mp4' '/content/islNTvideos/1-Thessalonians/3 Ch/V- 11-13.MP4'

In [ ]:
mv "/content/islNTvideos/Acts/Ch 27/Ch 27, V=-18-20.MP4" "/content/islNTvideos/Acts/Ch 27/Ch 27, V=18-20.MP4"

In [ ]:
ls /content/islNTvideos/*/*/*.MP4 | wc

    870    1730   41681


## Inspect Data

In [ ]:
from IPython.display import Video

# Replace with the actual path to one of your downloaded MP4 files
video_path = '/content/islNTvideos/1-Timothy/Ch- 1/v- 1-2 (Re-sign).MP4'
display(Video(video_path, embed=True, width=480)) # Assuming original width is around 960, setting to 50%

In [ ]:
video_path = '/content/islNTvideos/Philemon/Ch-1/1-3.MP4'
display(Video(video_path, embed=True, width=480))

In [ ]:
import os
import cv2
import glob

input_dir = '/content/islNTvideos'
output_dir = '/content/video_frames'
os.makedirs(output_dir, exist_ok=True)

video_files = glob.glob(f'{input_dir}/*/*/*.MP4') + glob.glob(f'{input_dir}/*/*/*.mp4')

for video_path in video_files:
    try:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error opening video file: {video_path}")
            continue

        # Get the total number of frames
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Check if the video has at least 200 frames
        if total_frames < 200:
            print(f"Video has less than 200 frames, skipping: {video_path}")
            cap.release()
            continue

        # Set the frame position to the 200th frame (0-indexed)
        cap.set(cv2.CAP_PROP_POS_FRAMES, 199)

        ret, frame = cap.read()

        if ret:
            # Resize the frame to 128x128
            resized_frame = cv2.resize(frame, (128, 128))

            # Create the output filename
            book_chapter = video_path.replace(input_dir + '/', '').replace('.MP4', '').replace('.mp4', '')
            output_filename = f"{book_chapter.replace('/', '_')}_frame_200.jpg"
            output_path = os.path.join(output_dir, output_filename)

            # Save the frame
            cv2.imwrite(output_path, resized_frame)
            print(f"Saved frame 200 from {video_path} to {output_path}")
        else:
            print(f"Could not read frame 200 from {video_path}")

        cap.release()

    except Exception as e:
        print(f"An error occurred processing {video_path}: {e}")

print("Finished processing videos.")

In [ ]:
ls /content/video_frames/* |wc

    870    1730   51251


In [ ]:
from IPython.display import Image
Image(filename='/content/video_frames/1-Timothy_Ch- 1_v- 15-20 (Re-sign)_frame_200.jpg')

In [ ]:
Image(filename='/content/video_frames/Philemon_Ch-1_8-11_frame_200.jpg')

In [ ]:
import os
import glob
from IPython.display import display, clear_output
from ipywidgets import Button, VBox, Layout, Image
from google.colab import output

image_dir = '/content/video_frames'
image_files = glob.glob(f'{image_dir}/*.jpg')

# Group images by book name
image_groups = {}
for image_path in image_files:
    filename = os.path.basename(image_path)
    book_name = filename.split('_')[0]
    if book_name not in image_groups:
        image_groups[book_name] = []
    image_groups[book_name].append(image_path)

book_names = list(image_groups.keys())
book_names.sort() # Sort book names alphabetically
current_book_index = 0

def display_book_images(book_name):
    clear_output(wait=True)
    print(f"Book: {book_name}")
    images = image_groups[book_name]
    # Display images in a grid-like manner (adjust columns as needed)
    # Create ipywidgets.Image instances
    image_widgets = [Image(value=open(img, 'rb').read(), width=128, height=128) for img in images]
    for i in range(0, len(image_widgets), 8): # Display 8 images per row
        row_images = image_widgets[i:i+8]
        display(VBox(row_images, layout=Layout(flex_flow='row wrap')))

    # Display the button after showing images for the current book
    if current_book_index < len(book_names) - 1:
        button = Button(description="Next Book")
        button.on_click(on_button_clicked)
        display(button)
    else:
        print("Finished displaying all books.")


def on_button_clicked(b):
    global current_book_index
    current_book_index += 1
    if current_book_index < len(book_names):
        display_book_images(book_names[current_book_index])


if book_names:
    display_book_images(book_names[current_book_index])

else:
    print("No images found in the specified directory.")

## Set up code

In [ ]:
! mkdir /content/logs

In [ ]:
! mkdir /content/temp

In [ ]:
! mkdir /content/dataprep_output

In [ ]:
! git clone https://github.com/kavitharaju/sign-bibles-dataset.git

In [ ]:
cd /content/sign-bibles-dataset/dataprep/isl/

/content/sign-bibles-dataset/dataprep/isl


In [ ]:
! ls /content/sign-bibles-dataset/dataprep/isl

In [ ]:
!git checkout -b split-cpu-pipeline-into-local-n-cloud

Switched to a new branch 'split-cpu-pipeline-into-local-n-cloud'


In [ ]:
!apt-get update && apt-get install -y parallel

In [ ]:
!parallel --version

GNU parallel 20210822
Copyright (C) 2007-2021 Ole Tange, http://ole.tange.dk and Free Software
Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <https://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
GNU parallel comes with no warranty.

Web site: https://www.gnu.org/software/parallel

When using programs that use GNU Parallel to process data for publication
please cite as described in the manpage.


In [ ]:
!git pull origin split-cpu-pipeline-into-local-n-cloud

In [ ]:
! pip install -r requirements-CPU-only.txt

In [ ]:
! git log

## Run data processing

In [ ]:
! python video_id_list.py /content/islNTvideos/ 1114 > /content/input_list_full.txt

python3: can't open file '/content/video_id_list.py': [Errno 2] No such file or directory


In [ ]:
!sed -n '361,870p' /content/input_list_full.txt > /content/input_list.txt

In [ ]:
! tail /content/input_list.txt

In [ ]:
! cat /content/input_list.txt | wc

    510    1673   27288


In [ ]:
! time(bash run_parallel_cpu.sh)

In [ ]:
cat /content/logs/success.log

1118	/content/islNTvideos/2-Peter/CH-3/8-10 Re.MP4
1115	/content/islNTvideos/2-Peter/CH-3/11-14.MP4
1116	/content/islNTvideos/2-Peter/CH-3/1-2.MP4
1124	/content/islNTvideos/2-Peter/CH-1/16-21.MP4
1123	/content/islNTvideos/2-Peter/CH-1/1-2.MP4
1120	/content/islNTvideos/2-Peter/CH-3/3-7.MP4
1127	/content/islNTvideos/2-Peter/CH-2/4-9 Re.MP4
1119	/content/islNTvideos/2-Peter/CH-3/8-10.MP4
1122	/content/islNTvideos/2-Peter/CH-1/10-15.MP4
1117	/content/islNTvideos/2-Peter/CH-3/15-18.MP4
1126	/content/islNTvideos/2-Peter/CH-2/10-14 Re.MP4
1130	/content/islNTvideos/2-Peter/CH-2/10-14.MP4
1121	/content/islNTvideos/2-Peter/CH-1/3-9.MP4
1131	/content/islNTvideos/2-Peter/CH-2/1-3.MP4
1139	/content/islNTvideos/1-John/CH-1/1-4.MP4
1140	/content/islNTvideos/1-John/CH-1/8-10.MP4
1134	/content/islNTvideos/1-John/CH-3/7-10.MP4
1133	/content/islNTvideos/1-John/CH-3/16-18.MP4
1128	/content/islNTvideos/2-Peter/CH-2/20-22.MP4
1136	/content/islNTvideos/1-John/CH-3/19-24.MP4
1129	/content/islNTvideos/2-Peter/

## Upload outputs

In [ ]:
pwd

'/content/sign-bibles-dataset/dataprep/isl'

In [ ]:
! ls /content/dataprep_output | wc

   2040    2040   33660


In [ ]:
! aws s3 cp /content/dataprep_output/ s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ --recursive --endpoint-url=https://files.us-east-3.lambda.ai  #--exclude "*" --include "*.npz"

upload: ../../../dataprep_output/1476.json to s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/1476.json
upload: ../../../dataprep_output/1476.transcripts.json to s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/1476.transcripts.json
upload: ../../../dataprep_output/1475.json to s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/1475.json
upload: ../../../dataprep_output/1475.transcripts.json to s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/1475.transcripts.json
upload: ../../../dataprep_output/1477.json to s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/1477.json
upload: ../../../dataprep_output/1475.mp4 to s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/1475.mp4
upload: ../../../dataprep_output/1477.pose-mediapipe.pose to s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/1477.pose-mediapipe.pose
upload: ../../../dataprep_output/1477.transcripts.json to s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/1477.transcripts.json
upload: ../../../dataprep_output/1478.json to s3://6

In [ ]:
! aws s3 ls s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ --endpoint-url=https://files.us-east-3.lambda.ai | wc

   3480   13920  165300


In [ ]:
! rm  /content/dataprep_output/*

## Post dataprep analysis

In [ ]:
! aws s3 cp s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ /content/jsons/  --recursive --endpoint-url=https://files.us-east-3.lambda.ai  --exclude "*" --include "*.json" --exclude "*transcript*"

In [ ]:
rm /content/jsons/*

In [ ]:
ls /content/jsons/ | wc

    870     870    8700


In [ ]:
! python duration_calculation.py /content/jsons/

args.directories=['/content/jsons/']
Total number of videos : 870
total_seconds=69927.98000000001
Total duration: 19 hours, 25 minutes, 27 seconds
Total number of unique verses: 3201


In [ ]:
import json
from pathlib import Path

verses = []
for json_file in Path("/content/jsons/").glob("*.json"):
    with open(json_file, "r") as f:
        data = json.load(f)
        ref = data["bible-ref"]
        vrefs = data["biblenlp-vref"]
        id = json_file.name.split("/")[-1].split(".")[0]
        for vref in vrefs:
            verses.append((ref,vref, id))
verses = sorted(verses, key=lambda x: x[1])
for item in verses:
  print(f"{item[1]}   {item[0]}   {item[2]}")

